# Condition Variables

In multithreaded programs, we often want to have one thread wait for some condition to be true before it can continue execution.

We might need one thread wait for some other thread to complete some action before it can continue execution.

We accomplish this using **condition variables**.

A **condition variable** is a queue of waiting threads.

We always need a lock associated with a condition variable. We have to grab the lock before using the condition variable.

Condition Variable Functions:

- `wait(cond *cv, lock *mutex)`
    - it is assumed that the thread calling wait holds the lock
    - adds thread to condition variable's queue, releases the lock, and puts the caller to sleep
    - When woken, wait will reacquire the lock and then return
- `signal(cond *cv)`
    - signals that one thread waiting on this condition variable can wake up
    - if no threads are waiting, it returns without doing anything


# Example: Implementing Join

```C
int done  = 0;
pthread_mutex_t m = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t c  = PTHREAD_COND_INITIALIZER;

void thr_exit() {
    Pthread_mutex_lock(&m);
    done = 1;
    Pthread_cond_signal(&c);
    Pthread_mutex_unlock(&m);
}

void *child(void *arg) {
    printf("child\n");
    thr_exit();
    return NULL;
}

void thr_join() {
    Pthread_mutex_lock(&m);
    while (done == 0)
        Pthread_cond_wait(&c, &m);
    Pthread_mutex_unlock(&m);
}

int main(int argc, char *argv[]) {
    printf("parent: begin\n");
    pthread_t p;
    Pthread_create(&p, NULL, child, NULL);
    thr_join();
    printf("parent: end\n");
    return 0;
}
```

Do we need the done variable?

```C

void thr_exit() {
    Pthread_mutex_lock(&m);
    Pthread_cond_signal(&c);
    Pthread_mutex_unlock(&m);
}

void thr_join() {
    Pthread_mutex_lock(&m);
    Pthread_cond_wait(&c, &m);
    Pthread_mutex_unlock(&m);
}
```

While using condition variables is a little complicated, there are some best practices which will make your life much easier:

1) always recheck the condition after calling on wait
   - the state of the condition might have changed between being woken and reaquiring the lock.
2) Always grab the lock before calling on `wait()` and release it ASAP after
3) Shouls always grab the lock before `signal()` and release it ASAP after

# The Producer/Consumer Model

The producer adds information to some shared buffer.

The consumer retrieves information from the shared buffer.

## Practical Example: A Web Server!

The producer accepts and adds web requests to a shared queue.

The consumers remove, process, and respond to the web requests.

### Condition Variables

Condition Variables are used to ensure that there is information in the buffer for a consumer to consume (otherwise consumers `wait()` for the buffer to hold information).

Condition variables are also used to ensure that the queue is not full.

If the buffer is empty, consumers will `wait()` until signaled by producers to continue.

If the bugger is full, producers will `wait()` until signaled by the consumers to continue.

## A second example: Shell Pipes

The pipe on the shell uses the producer/consumer model:

`cat test.txt | wc -l`

Pipe the output of `cat test.txt` into the input of `wc`

`wc` must wait until there is input in its buffer before it can start executing.